# Testing 5-bit trigger codes w/ arduino & pyserial

In [1]:
import serial
import time

### CONFIG

In [2]:
# CHANGE THIS to match your Arduino port:
#   Windows: "COM3", "COM4", ...
#   macOS:   "/dev/tty.usbmodemXXXX" or "/dev/tty.usbserialXXXX"
#   Linux:   "/dev/ttyACM0", "/dev/ttyUSB0", etc.
SERIAL_PORT = "/dev/ttyACM0"
BAUD_RATE = 115200


## Functions

In [3]:

def open_trigger_serial(port=SERIAL_PORT, baud_rate=BAUD_RATE):
    ser = serial.Serial(port, baud_rate, timeout=1)
    # Arduino auto-resets when serial opens; give it a moment
    time.sleep(2.0)
    
    print(f"Opened serial port {port} at {baud_rate} baud.")
    
    return ser

def send_trigger(ser, code):
    """
    Send a trigger code in [0, 31] to Arduino.
    Arduino maps the 5 LSBs to pins (4, 6, 8, 10, 12)
    and pulses them for PULSE_MS ms.
    """
    if not (0 <= code <= 31):
        raise ValueError("Trigger code must be between 0 and 31 (inclusive).")

    ser.write(bytes([code]))
    ser.flush()   # push out immediately

## Explanation:
OpenBCI board has 5 input pins resulting in 32 (2^5) possible trigger codes (0-31). Each code is represented by the binary state of the 5 pins. For example:
- Code 0: 00000 (all pins LOW)
- Code 1: 00001 (pin 1 HIGH, others LOW)
- Code 31: 11111 (all pins HIGH)

The arduino is programmed to take an input code (0-32) via serial and set the corresponding pins HIGH/LOW based on the binary representation of that code.

## Tests

In [ ]:
ser = open_trigger_serial()

for _ in range(2):
    for code in range(0, 32):
        print(f"Sending trigger code: {code} -> {code:05b}") # (bytes: {bytes([code])})")
        send_trigger(ser, code)
        time.sleep(0.5)
        
    time.sleep(5)
        
ser.close()